<a href="https://colab.research.google.com/github/J3sus4lmazan/Analisis-Energetico-en-Mexico/blob/main/FirstPart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Conexion a base de datos local
import mysql.connector
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import geopandas as gpd
import folium
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVC #Support Vector Classifier
from sklearn.pipeline import make_pipeline

In [ ]:
cnx = mysql.connector.connect(
    host="localhost",
    port=3306,
    user="root",
    password='hola20',
    database='miproyecto'
)
cursor = cnx.cursor()

In [ ]:
#Se obtienen la informacion de la tabla generacion bruta por tecnologia
cursor.execute("SELECT * FROM genbrutatech")
GBT = cursor.fetchall() #Generacion bruta por tecnologia
GBTdf = pd.DataFrame(GBT, columns=['GBT_ID', 'Month_Year', 'EnergyTypeID', 'MegaWatt_hourTotal', 'CapMWattPerTech'])
GBTdf = GBTdf.set_index('GBT_ID', drop=True)
GBTdf.head()

In [ ]:
#Se obtienen la informacion de la tabla Meses
cursor.execute("SELECT * FROM months")
M = cursor.fetchall() #Meses
Mdf = pd.DataFrame(M, columns=['Month_YearID', 'MonthDescription','Y34R'])
Mdf = Mdf.set_index('Month_YearID', drop=True)
Mdf.head()

In [ ]:
#Se obtienen la informacion de tipo de energia
cursor.execute("SELECT * FROM energytypes")
ET = cursor.fetchall() #Tipo de energia
ETdf = pd.DataFrame(ET, columns=['EnergyTypeID', 'EnergyTypeName'])
ETdf = ETdf.set_index('EnergyTypeID', drop=True)
ETdf

In [ ]:
#Se obtienen la informacion de tipo de energia
cursor.execute("SELECT * FROM energytypes")
ET = cursor.fetchall() #Tipo de energia
ETdf = pd.DataFrame(ET, columns=['EnergyTypeID', 'EnergyTypeName'])
ETdf = ETdf.set_index('EnergyTypeID', drop=True)
ETdf

In [ ]:
#Se unen las 3 tablas
GBT_full = pd.merge(GBTdf, Mdf, left_on='Month_Year', right_index=True).sort_index()
GBT_full = pd.merge(GBT_full, ETdf, left_on='EnergyTypeID', right_index=True).sort_index()
GBT_full = GBT_full.dropna(axis = 0, how = 'any') #Se retiran los NaN
GBT_full = GBT_full.drop(columns=['Month_Year', 'EnergyTypeID']) #Se retiran las columnas con valores referenciales
GBT_full

In [ ]:
#Se obtiene toda la informacion por tipo de tecnologia
for_Termo = GBT_full['EnergyTypeName'] == 'Termoeléctrica'
for_Dual = GBT_full['EnergyTypeName'] == 'Dual'
for_Carbo = GBT_full['EnergyTypeName'] == 'Carboeléctrica'
for_Nucleo = GBT_full['EnergyTypeName'] == 'Nucleoeléctrica'
for_Geo = GBT_full['EnergyTypeName'] == 'Geotérmica'
for_Eolica = GBT_full['EnergyTypeName'] == 'Eólica'
for_Hidro = GBT_full['EnergyTypeName'] == 'Hidroeléctrica'
for_Foto = GBT_full['EnergyTypeName'] == 'Fotovoltaica'
GBT_Termo = GBT_full[for_Termo]
GBT_Dual = GBT_full[for_Dual]
GBT_Carbo = GBT_full[for_Carbo]
GBT_Nucleo = GBT_full[for_Nucleo]
GBT_Geo = GBT_full[for_Geo]
GBT_Eolica = GBT_full[for_Eolica]
GBT_Hidro = GBT_full[for_Hidro]
GBT_Foto = GBT_full[for_Foto]
GBT_Foto

In [ ]:
#Se obtiene la informacion de la energia generada por estado
cursor.execute("SELECT GBS_ID, MegaWatt_hourTotal, MonthDescription, Y34R, StateName FROM genbruperstate JOIN months ON genbruperstate.Month_YearID = months.Month_YearID JOIN states ON states.State_ID = genbruperstate.State_ID;") 
               #genbruperstate.Month_YearID, genbruperstate.State_ID  columnas con valores referenciales
GBS = cursor.fetchall() #Generacion bruta por estado
GBSdf = pd.DataFrame(GBS, columns=['GBS_ID', 'MegaWatt_hourTotal', 'MonthDescription', 'Year','StateName'])
#, 'Month_YearID', 'State_ID'
GBSdf = GBSdf.set_index('GBS_ID', drop=True)
GBSdf = GBSdf.dropna(axis = 0, how = 'any') #Se retiran los NaN
GBSdf.head()

In [ ]:
#Se obtiene la informacion de la tabla de capacidad efectiva
cursor.execute("SELECT * FROM effectivecap")
EFC = cursor.fetchall() #Capacidad efectiva
EFCdf = pd.DataFrame(EFC, columns=['EFC_ID', 'MegaWatt', 'Month_YearID', 'State_ID'])
EFCdf = EFCdf.set_index('EFC_ID', drop=True)
#Se obtiene la informacion de la tabla de estados
cursor.execute("SELECT * FROM states")
S = cursor.fetchall() #Estados
Sdf = pd.DataFrame(S, columns=['State_ID', 'StateName'])
Sdf = Sdf.set_index('State_ID', drop=True)
#Se combinan las 3 tablas
EFC_full = pd.merge(EFCdf, Mdf, left_on='Month_YearID', right_index=True).sort_index()
EFC_full = pd.merge(EFC_full, Sdf, left_on='State_ID', right_index=True).sort_index()
EFC_full = EFC_full.dropna(axis = 0, how = 'any') #Se retiran los NaN
EFC_full = EFC_full.drop(columns=['Month_YearID', 'State_ID']) #Se retiran las columnas con valores referenciales
EFC_full.head()

In [ ]:
#Se obtiene la informacion de la tabla de comercio interior
cursor.execute("SELECT * FROM comerciointer")
CI = cursor.fetchall() #Comercio interior
CIdf = pd.DataFrame(CI, columns=['CI_ID', 'MegaWatt_hour', 'QuantityUsers', 'Month_YearID', 'Sector', 'CentsPerKW_hour'])
CIdf = CIdf.set_index('CI_ID', drop=True)
#Se combinan con la tabla de meses
CI_full = pd.merge(CIdf, Mdf, left_on='Month_YearID', right_index=True).sort_index()
CI_full = CI_full.dropna(axis = 0, how = 'any') #Se retiran los NaN
CI_full = CI_full.drop(columns=['Month_YearID']) #Se retiran las columnas con valores referenciales
CI_full.head()

In [ ]:
#Se obtiene toda la informacion por sector
for_Res = CI_full['Sector'] == 'Residencial'
for_Com = CI_full['Sector'] == 'Comercial'
for_Serv = CI_full['Sector'] == 'Servicios'
for_Agr = CI_full['Sector'] == 'Agricola'
for_Emp_Med = CI_full['Sector'] == 'Empresa Mediana'
for_G_Ind = CI_full['Sector'] == 'Gran Industria'
CI_Res = CI_full[for_Res]
CI_Com = CI_full[for_Com]
CI_Serv = CI_full[for_Serv]
CI_Agr = CI_full[for_Agr]
CI_Emp_Med = CI_full[for_Emp_Med]
CI_G_Ind = CI_full[for_G_Ind]
CI_G_Ind.head()

In [ ]:
#Se obtiene la informacion de la tabla de comercio exterior
cursor.execute("SELECT * FROM comercioexter")
CE = cursor.fetchall() #Comercio exterior
CEdf = pd.DataFrame(CE, columns=['CE_ID', 'MegaWatt_hour', 'Month_YearID', 'Country', 'IMP_EXP'])
CEdf = CEdf.set_index('CE_ID', drop=True)
#Se combinan con la tabla de meses
CE_full = pd.merge(CEdf, Mdf, left_on='Month_YearID', right_index=True).sort_index()
CE_full = CE_full.dropna(axis = 0, how = 'any') #Se retiran los NaN
CE_full = CE_full.drop(columns=['Month_YearID']) #Se retiran las columnas con valores referenciales
CE_full.head()

In [ ]:
#Se separa la informacion de energia importada y exportada
for_impor = CE_full['IMP_EXP'] == 'Importada'
for_exp = CE_full['IMP_EXP'] == 'Exportada'
CE_impor = CE_full[for_impor]
CE_exp = CE_full[for_exp]
CE_impor.head()

In [ ]:
#¿Cuanta diferencia de energia producida por termoelectica existe con respecto a la fotovoltaica y eolica?
Q1 = GBT_full.groupby('EnergyTypeName').sum()
total_termoelectrica = Q1.iloc[7, 0]
total_fotovoltaica = Q1.iloc[3,0]
total_eolica = Q1.iloc[2,0]
diferencia_1 = total_termoelectrica / total_fotovoltaica
diferencia_2 = total_termoelectrica / total_eolica
print("Se produce", diferencia_1, "veces mas energia termoelectrica que fotovoltaica", "\n", "Se produce", diferencia_2, "veces mas energia termoelectrica que eolica")

In [ ]:
#¿Cuales son los 3 principales estados que más producen energia?
Q2 = GBSdf.groupby('StateName').sum()
Q2 = Q2.sort_values('MegaWatt_hourTotal', ascending=False)
Q2 = Q2.iloc[0:3]
Q2 / 1000000 #cantidad en millon por megawatt hora

In [ ]:
#¿Cual es el estado con mayor capacidad efectiva en la actualidad?
Q3 = EFC_full.groupby('StateName').sum()
Q3 = Q3.sort_values('MegaWatt', ascending=False)
Q3 = Q3.iloc[0:3]
Q3

In [ ]:
#¿Cuál es el sector que más consumo de energía realiza?
Q4 = CI_full.groupby('Sector').sum()
Q4 = Q4.sort_values('MegaWatt_hour', ascending = False)
Q4.iloc[0]

In [ ]:
#¿Cuál es el promedio de energia que se exporta a Estados Unidos?
Q5 = CE_full.groupby('Country').mean()
Q5 = Q5[Q5.index == 'Estados Unidos']
Q5

In [ ]:
#Se cierra el cursor
cursor.close()
#TABLAS
#GBT_full   Generacion bruta por tipo de tecnologia
#GBSdf      Generacion bruta por estado
#CI_full    Comercio Interno
#CE_full    Comercio exterior

In [ ]:
#Se grafica la energia generada
GBT_by_year = GBT_full.groupby('Y34R')['MegaWatt_hourTotal'].mean()
Grafica_GBT_0 = GBT_by_year.plot(kind='bar',cmap='ocean')
plt.ylabel('Mega Watts hora')
plt.xlabel('Año')
plt.title('Promedio de Energia generada 2005-2019');

In [ ]:
#Grafica de caja
N = GBT_full.groupby('MonthDescription').sum()
sns.set(style="whitegrid");

sns.boxplot(x=N['MegaWatt_hourTotal']);
plt.axvline(x=N['MegaWatt_hourTotal'].mean(), c='y');

In [ ]:
#Grafica de energia solar anual
GBT_foto_by_year = GBT_Foto.groupby('Y34R')['MegaWatt_hourTotal'].sum()
Grafica_GBT_1 = GBT_foto_by_year.plot(kind='bar',cmap='YlOrRd')
plt.ylabel('Mega Watts hora')
plt.xlabel('Año')
plt.title('Total de Energia solar 2005-2019');

In [ ]:
#Grafica de energia termoelectrica anual
GBT_termo_by_year = GBT_Termo.groupby('Y34R')['MegaWatt_hourTotal'].sum()
Grafica_GBT_2 = GBT_termo_by_year.plot(kind='bar',cmap='Set1')
plt.ylabel('Mega Watts hora')
plt.xlabel('Año')
plt.title('Total de Energia Termoelectrica 2005-2019');

In [ ]:
#Grafica de energia Dual anual
GBT_dual_by_year = GBT_Dual.groupby('Y34R')['MegaWatt_hourTotal'].sum()
Grafica_GBT_3 = GBT_dual_by_year.plot(kind='bar',cmap='Accent')
plt.ylabel('Mega Watts hora')
plt.xlabel('Año')
plt.title('Total de Energia Dual 2005-2019');

In [ ]:
#Grafica de energia Carboelectrica anual
GBT_carbo_by_year = GBT_Carbo.groupby('Y34R')['MegaWatt_hourTotal'].sum()
Grafica_GBT_4 = GBT_carbo_by_year.plot(kind='bar',cmap='Pastel1')
plt.ylabel('Mega Watts hora')
plt.xlabel('Año')
plt.title('Total de Energia Carboelectrica 2005-2019');

In [ ]:
#Grafica de energia Nucleoelectrica anual
GBT_nucleo_by_year = GBT_Nucleo.groupby('Y34R')['MegaWatt_hourTotal'].sum()
Grafica_GBT_5 = GBT_nucleo_by_year.plot(kind='bar',cmap='Pastel2')
plt.ylabel('Mega Watts hora')
plt.xlabel('Año')
plt.title('Total de Energia Nucleoelectrica 2005-2019');

In [ ]:
#Grafica de energia Geotermica anual
GBT_geo_by_year = GBT_Geo.groupby('Y34R')['MegaWatt_hourTotal'].sum()
Grafica_GBT_6 = GBT_geo_by_year.plot(kind='bar',cmap='PuOr')
plt.ylabel('Mega Watts hora')
plt.xlabel('Año')
plt.title('Total de Energia Geotermica 2005-2019');

In [ ]:
#Grafica de energia Eolica anual
GBT_eolica_by_year = GBT_Eolica.groupby('Y34R')['MegaWatt_hourTotal'].sum()
Grafica_GBT_7 = GBT_eolica_by_year.plot(kind='bar',cmap='RdGy')
plt.ylabel('Mega Watts hora')
plt.xlabel('Año')
plt.title('Total de Energia Eolica 2005-2019');

In [ ]:
#Grafica de energia Hidroelectrica anual
GBT_hidro_by_year = GBT_Hidro.groupby('Y34R')['MegaWatt_hourTotal'].sum()
Grafica_GBT_8 = GBT_hidro_by_year.plot(kind='bar',cmap='cool')
plt.ylabel('Mega Watts hora')
plt.xlabel('Año')
plt.title('Total de Energia Hidroelectrica 2005-2019');

In [ ]:
#Grafica de proporciones
GBT_by_energy = GBT_full.groupby('EnergyTypeName')['MegaWatt_hourTotal'].sum()
plt.figure(figsize=(10,8))
Grafica_2_GBT = GBT_by_energy.plot(kind='pie',cmap='Paired')
plt.ylabel('')
plt.xlabel('Tipos de energia')
plt.title('Energia generada 2005-2019 por tipo de tecnologia');

In [ ]:
#Preparacion de datos para mapa cloropletico
for_planta_movil = EFC_full['StateName'] != 'Plantas Móviles'
EFC_sin_pm = EFC_full[for_planta_movil]
EFC_by_year = EFC_sin_pm.groupby('StateName')['MegaWatt'].sum()
EFC_for_map = EFC_by_year.to_frame()
EFC_for_map = EFC_for_map.rename_axis('index').reset_index()
EFC_for_map

#Se retiran los acentos para homologar el json con las coordenadas geograficas
EFC_for_map = EFC_for_map.replace({
    'Michoacán': 'Michoacan',
    'México': 'Mexico',
    'Nuevo León': 'Nuevo Leon',
    'Querétaro': 'Queretaro',
    'San Luis Potosí': 'San Luis Potosi',
    'Yucatán': 'Yucatan'
})

EFC_for_map.head()

In [ ]:
#Se preparan los datos con las coodenadas
df_geo = gpd.read_file('../Procesamiendo de datos/mexico2.json')
df_geo = df_geo.merge(EFC_for_map, left_on='name', right_on='index')
geojson = '../Procesamiendo de datos/mexico2.json'
df_geo

In [ ]:
## Mapa cloropletico
mapa = folium.Map(location=[24.5, -99], zoom_start=4.5)

folium.Choropleth(
    geo_data=geojson,
    data=EFC_for_map,
    columns=['index', 'MegaWatt'],
    key_on='feature.properties.name',
    fill_color='YlGnBu',
).add_to(mapa)

mapa

In [ ]:
#Grafica de energia consumida por sector
CI_by_sector = CI_full.groupby('Sector')['MegaWatt_hour'].sum()
plt.figure(figsize=(10,8))
Grafica_1_CI = CI_by_sector.plot(kind='pie',cmap='Paired')
plt.ylabel('')
plt.xlabel('Sectores')
plt.title('Energia consumida por sector 2005-2019');

In [ ]:
#Proyecciones con regresion lineal
#Aqui elegiremos 3 campos: Cantidad de energia consumida, cantidad de usuarios y año. Dando como resultado 
#una proyeccion del costo en centavos por KW hora.

# Lista de proyecciones
#CI_Res = residencial 
#CI_Emp_Med = empresa mediana
#CI_G_Ind = Gran industria

#Proyeccion sector residencial
Xr = CI_Res[['MegaWatt_hour', 'QuantityUsers','Y34R']]
Yr = CI_Res['CentsPerKW_hour']

In [ ]:
#Dividir los datos en Train y Test
Xr_train, Xr_test, Yr_train, Yr_test = train_test_split(Xr, Yr, test_size = 0.4, random_state=5)
print(Xr_train.shape)
print(Xr_test.shape)
print(Yr_train.shape)
print(Yr_test.shape)

In [ ]:
#Entrenamiento y prediccion
lin_model = LinearRegression()
lin_model.fit(Xr_train, Yr_train)

In [ ]:
yr_train_predict = lin_model.predict(Xr_train)
MSE = mean_squared_error(Yr_train,yr_train_predict)
print("Entrenamiento: MSE ="+str(MSE))

yr_test_predict = lin_model.predict(Xr_test)
MSE = (mean_squared_error(Yr_test, yr_test_predict))
print("Pruebas: MSE ="+str(MSE))

In [ ]:
#Comparacion de predicciones
dfr_predicciones = pd.DataFrame({'valor_real':Yr_test, 'prediccion':yr_test_predict})
dfr_predicciones = dfr_predicciones.reset_index(drop = True)
dfr_predicciones.head(10)

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(dfr_predicciones,"-")
plt.xlabel("Time")
plt.ylabel("Value")
plt.grid(True)

In [ ]:
#Proyeccion empresa mediana
Xe = CI_Emp_Med[['MegaWatt_hour', 'QuantityUsers','Y34R']]
Ye = CI_Emp_Med['CentsPerKW_hour']

In [ ]:
Xe_train, Xe_test, Ye_train, Ye_test = train_test_split(Xe, Ye, test_size = 0.4, random_state=5)
print(Xe_train.shape)
print(Xe_test.shape)
print(Ye_train.shape)
print(Ye_test.shape)

In [ ]:
lin_model = LinearRegression()
lin_model.fit(Xe_train, Ye_train)

In [ ]:
ye_train_predict = lin_model.predict(Xe_train)
MSE = mean_squared_error(Ye_train,ye_train_predict)
print("Entrenamiento: MSE ="+str(MSE))

ye_test_predict = lin_model.predict(Xe_test)
MSE = (mean_squared_error(Ye_test, ye_test_predict))
print("Pruebas: MSE ="+str(MSE))

In [ ]:
dfe_predicciones = pd.DataFrame({'valor_real':Ye_test, 'prediccion':ye_test_predict})
dfe_predicciones = dfe_predicciones.reset_index(drop = True)
dfe_predicciones.head(10)

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(dfe_predicciones,"-")
plt.xlabel("Time")
plt.ylabel("Value")
plt.grid(True)

In [ ]:
#Proyeccion gran industria
Xg = CI_G_Ind[['MegaWatt_hour', 'QuantityUsers','Y34R']]
Yg = CI_G_Ind['CentsPerKW_hour']

In [ ]:
Xg_train, Xg_test, Yg_train, Yg_test = train_test_split(Xg, Yg, test_size = 0.4, random_state=5)
print(Xg_train.shape)
print(Xg_test.shape)
print(Yg_train.shape)
print(Yg_test.shape)

In [ ]:
lin_model = LinearRegression()
lin_model.fit(Xg_train, Yg_train)

In [ ]:
yg_train_predict = lin_model.predict(Xg_train)
MSE = mean_squared_error(Yg_train,yg_train_predict)
print("Entrenamiento: MSE ="+str(MSE))

yg_test_predict = lin_model.predict(Xg_test)
MSE = (mean_squared_error(Yg_test, yg_test_predict))
print("Pruebas: MSE ="+str(MSE))

In [ ]:
dfg_predicciones = pd.DataFrame({'valor_real':Yg_test, 'prediccion':yg_test_predict})
dfg_predicciones = dfg_predicciones.reset_index(drop = True)
dfg_predicciones.head(10)

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(dfg_predicciones,"-")
plt.xlabel("Time")
plt.ylabel("Value")
plt.grid(True)